# Scenario 1:  Find Number of Records Indexed

## Redis

In [2]:
! redis-cli -p 12000 -3 FT.SEARCH locationidx \* LIMIT 0 0

1# attributes => (empty array)
2# warning => (empty array)
3# total_results => (integer) 1000000
4# format => STRING
5# results => (empty array)


## Pinot

```sql
SELECT COUNT(*) AS Count
FROM locations;
```

In [3]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s1.json http://localhost:8099/query/sql | jq '.resultTable'

# Scenario 2:  Display 1 Sample Record

## Redis

In [ ]:
! redis-cli -p 12000 -3 FT.SEARCH locationidx \* LIMIT 0 1

1# attributes => (empty array)
2# warning => (empty array)
3# total_results => (integer) 1000000
4# format => STRING
5# results => 
   1) 1# id => "location:b9ec8654-a55c-490e-a372-e78143f1d623"
      2# extra_attributes => 1# "$" => "{\"uuid\":\"b9ec8654-a55c-490e-a372-e78143f1d623\",\"point_lnglat\":\"-88.45195 42.12357\",\"point_wkt\":\"POINT (-88.45195 42.12357)\",\"dob\":1945}"
      3# values => (empty array)


## Pinot

```sql
SELECT *
FROM locations
LIMIT 1;
```

In [4]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s2.json http://localhost:8099/query/sql | jq '.resultTable'

# Find the UUIDs for the 3 closest users to Woodland Park, CO (-105.0810 38.9922) born in 2010 

## Redis

In [5]:
! redis-cli -p 12000 -3 FT.AGGREGATE locationidx '@dob:[2010 2010]' LOAD 3 @uuid @dob @point_lnglat APPLY 'geodistance(@point_lnglat, -105.0810, 38.9922) * 0.000621371' AS miles SORTBY 2 @miles ASC LIMIT 0 3

1# attributes => (empty array)
2# total_results => (integer) 3
3# format => STRING
4# results => 
   1) 1# extra_attributes => 
         1# "uuid" => "1c8571ab-679f-4b45-bbd3-3a62d201053f"
         2# "dob" => "2010"
         3# "point_lnglat" => "-105.08797 38.97149"
         4# "miles" => "1.47949677842"
      2# values => (empty array)
   2) 1# extra_attributes => 
         1# "uuid" => "16f90c6b-d1fa-40d0-a486-b47cb880420b"
         2# "dob" => "2010"
         3# "point_lnglat" => "-105.02237 38.99941"
         4# "miles" => "3.18842858488"
      2# values => (empty array)
   3) 1# extra_attributes => 
         1# "uuid" => "d3f09f54-f12a-4158-a028-b49981ea6e56"
         2# "dob" => "2010"
         3# "point_lnglat" => "-105.0723 39.07854"
         4# "miles" => "5.98544936315"
      2# values => (empty array)
5# warning => (empty array)


## Pinot

```sql
SELECT uuid,
  dob,
  STDistance(
    point_h3,
    toSphericalGeography(ST_GeomFromText('POINT (-105.0810 38.9922)'))
  ) / 1609.344 AS miles
FROM locations
WHERE (
    dob = 2010
  )
ORDER BY miles ASC
LIMIT 3;
```

In [6]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s3.json http://localhost:8099/query/sql | jq '.resultTable'

# Find the count of users within the State of Colorado

![Colorado](../data/co.png)

## Redis

In [7]:
! redis-cli -p 12000 -3 FT.SEARCH locationidx '@point_wkt:[WITHIN $Colorado]' PARAMS 2 Colorado 'POLYGON((-109.0448 37.0004,-102.0424 36.9949,-102.0534 41.0006,-109.0489 40.9996,-109.0448 37.0004,-109.0448 37.0004))' LIMIT 0 0 DIALECT 2

1# attributes => (empty array)
2# warning => (empty array)
3# total_results => (integer) 12500
4# format => STRING
5# results => (empty array)


## Pinot

```sql
SELECT COUNT(*) AS CO_Total
FROM locations
WHERE ST_Within(
    point_h3,
    toSphericalGeography(
      ST_GeomFromText(
        'POLYGON((-109.0448 37.0004,-102.0424 36.9949,-102.0534 41.0006,-109.0489 40.9996,-109.0448 37.0004,-109.0448 37.0004))'
      )
    )
  ) IS True;
```

In [8]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s4.json http://localhost:8099/query/sql | jq '.resultTable'

# Find the count of users by year of birth that are in either of the 2 polygons in the Woodland Park CO area.  Sort by year, ascending.

![WP](../data/wp.png)

## Redis

In [1]:
! redis-cli -p 12000 -3 FT.AGGREGATE locationidx '(@point_wkt:[WITHIN $WP1] | @point_wkt:[WITHIN $WP2])' PARAMS 4 WP1 'POLYGON((-105.080 38.980, -105.020 38.980, -105.020 38.920, -105.080 38.920, -105.080 38.980))' WP2 'POLYGON((-105.110 38.960, -105.050 38.960, -105.050 38.900, -105.110 38.900, -105.110 38.960))' LOAD 1 @dob GROUPBY 1 @dob REDUCE COUNT 0 AS Total SORTBY 2 @Total DESC DIALECT 2

1# attributes => (empty array)
2# total_results => (integer) 15
3# format => STRING
4# results => 
    1) 1# extra_attributes => 
          1# "dob" => "1966"
          2# "Total" => "2"
       2# values => (empty array)
    2) 1# extra_attributes => 
          1# "dob" => "1934"
          2# "Total" => "1"
       2# values => (empty array)
    3) 1# extra_attributes => 
          1# "dob" => "1965"
          2# "Total" => "1"
       2# values => (empty array)
    4) 1# extra_attributes => 
          1# "dob" => "1999"
          2# "Total" => "1"
       2# values => (empty array)
    5) 1# extra_attributes => 
          1# "dob" => "1941"
          2# "Total" => "1"
       2# values => (empty array)
    6) 1# extra_attributes => 
          1# "dob" => "1944"
          2# "Total" => "1"
       2# values => (empty array)
    7) 1# extra_attributes => 
          1# "dob" => "1961"
          2# "Total" => "1"
       2# values => (empty array)
    8) 1# extra_attributes => 
          1# "do

## Pinot

```sql
SELECT dob,
  COUNT(*) AS Total
FROM locations
WHERE ST_Within(
    point_h3,
    toSphericalGeography(
      ST_GeomFromText(
        'POLYGON((-105.080 38.980, -105.020 38.980, -105.020 38.920, -105.080 38.920, -105.080 38.980))'
      )
    )
  ) IS True
  OR ST_Within(
    point_h3,
    toSphericalGeography(
      ST_GeomFromText(
        'POLYGON((-105.110 38.960, -105.050 38.960, -105.050 38.900, -105.110 38.900, -105.110 38.960))'
      )
    )
  ) IS True
GROUP BY dob
ORDER BY Total DESC;
```

In [2]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s5.json http://localhost:8099/query/sql | jq '.resultTable'

{
  "dataSchema": {
    "columnNames": [
      "dob",
      "Total"
    ],
    "columnDataTypes": [
      "LONG",
      "LONG"
    ]
  },
  "rows": [
    [
      1966,
      2
    ],
    [
      1944,
      1
    ],
    [
      1961,
      1
    ],
    [
      1962,
      1
    ],
    [
      1964,
      1
    ],
    [
      1965,
      1
    ],
    [
      1941,
      1
    ],
    [
      1980,
      1
    ],
    [
      1986,
      1
    ],
    [
      1934,
      1
    ]
  ]
}
